<img src="../images/bentley_logo.png" alt="Bentley" style="margin-left: 0">
<p style="margin: 0; font-size: 12px; color: #666">
    CityPhi and its documentation, Copyright Â© 2024 Bentley Systems, Incorporated. All rights reserved.
</p>

## CityPhi Tutorial
This notebook provides a guided walkthrough of common CityPhi use cases on small data sets.
We recommend taking some time to explore/navigate in the CityPhi window after each step. Some notes:

- Left-click to pan / move
- Right-click to tilt and rotate
- Scroll wheel to zoom-in and zoom-out

This is a tutorial, it is not intended as a computational demo of CityPhi. 

<img src="../images/new_main.png" width="100%">

### Contents

<a href="#Start-CityPhi">Start CityPhi</a>

<a href="#Web-basemap">Web basemap<a>

<a href="#Simple-point-layer">Simple point layer</a>
- <a href="#PointFeature">PointFeature</a>
- <a href="#Layer">Layer</a>

<a href="#Layer-configuration">Layer configuration</a>
- <a href="#Layer-ordering">Layer ordering</a>
- <a href="#Layer-parameters-and-attributes">Layer parameters and attributes</a>
- <a href="#Stacking-points">Stacking points</a>
- <a href="#Adding-feature-attributes">Adding feature attributes<a>
- <a href="#Coloring">Coloring</a>
- <a href="#Sorting">Sorting</a>
- <a href="#Filtering">Filtering</a>

<a href="#Interactivity">Interactivity<a>

<a href="#Queries">Queries</a>

<a href="#Polylines">Polylines</a>
- <a href="#Animated-flow-on-polylines">Animated flow on polylines</a>
- <a href="#Extruded-polylines">Extruded polylines</a>

<a href="#Trajectories">Trajectories</a>
- <a href="#Animated-points">Animated points</a>
- <a href="#Query-by-screenline">Query by screenline</a>
- <a href="#Parabolic-trajectories">Parabolic trajectories</a>
- <a href="#Motion-trails">Motion trails</a>

<a href="#Polygons">Polygons</a>

## Start CityPhi

The CityPhi application must be imported (first cell). 

We also import some other libraries, and set up the data directory (second cell). 

The third cell launches the CityPhi application.
- Note: CityPhi window may open behind other windows

## Data Setup - MTC Model

This notebook uses MTC travel model data:
- **MAZ Zones**: Shapefile with MAZ boundaries and centroids (replacing parcels)
- **Individual Trips**: Trip records from CTRAMP model (replacing activities)
- **Time periods**: 30-minute periods (1-48) starting at 3:00 AM
- **Trip purposes**: Various activity types from the model

### Python Environment Setup

**Required Packages:**
- CityPhi (from EMME installation)
- pandas, numpy, geopandas (data processing)
- bokeh (visualization support)

**Setup Options:**

**Option 1: Create a dedicated environment (Recommended)**
```powershell
# Create and activate virtual environment
python -m venv cityphi_env
.\cityphi_env\Scripts\Activate.ps1

# Install dependencies
pip install -r ../requirements.txt

# Install CityPhi from your EMME installation
# Adjust path to match your EMME version
pip install "C:\Program Files\INRO\Emme\Emme 4.6.2\Python312-64\Lib\site-packages\cityphi_*"
```

**Option 2: Use existing environment**
If you already have CityPhi and the required packages installed, just select that kernel.

**To select kernel in VS Code:**
1. Click the kernel selector in the top right
2. Select "Python Environments"
3. Choose your Python interpreter with CityPhi installed

Run the cell below to verify your environment has all required packages:

In [ ]:
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")

# Check for required packages
required_packages = {
    'cityphi': 'CityPhi visualization (from EMME)',
    'geopandas': 'Geospatial data processing',
    'pandas': 'Data manipulation',
    'numpy': 'Numerical computing',
    'bokeh': 'Interactive visualization'
}

print("\n" + "="*60)
print("PACKAGE VERIFICATION")
print("="*60)

all_installed = True
for pkg, description in required_packages.items():
    try:
        mod = __import__(pkg)
        version = getattr(mod, '__version__', 'installed')
        print(f"✓ {pkg:12} {version:10} - {description}")
    except ImportError:
        print(f"✗ {pkg:12} NOT FOUND  - {description}")
        all_installed = False

print("="*60)
if all_installed:
    print("✓ All required packages are installed!")
else:
    print("✗ Some packages are missing. Install with: pip install -r requirements.txt")
    print("  (CityPhi must be installed separately from EMME)")

In [ ]:
import pandas
import numpy
import os
import geopandas as gpd
from IPython.display import display
from pathlib import Path

# ===== CONFIGURATION - UPDATE THESE PATHS FOR YOUR MACHINE =====
# Option 1: Use relative path from this notebook to data folder
notebook_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
data_dir = notebook_dir.parent / "data"

# Option 2: Or set absolute paths (uncomment and modify as needed)
# maz_shapefile_path = r"C:\path\to\your\cityphi_data_test"
# trip_data_path = r"C:\path\to\your\2023-tm22-dev-version-05\ctramp_output"

# Option 3: Use environment variables (recommended for team projects)
maz_shapefile_path = os.environ.get('CITYPHI_MAZ_DATA', str(data_dir / "maz_shapes"))
trip_data_path = os.environ.get('CITYPHI_TRIP_DATA', str(data_dir / "trips"))

print(f"MAZ Shapefile path: {maz_shapefile_path}")
print(f"Trip data path: {trip_data_path}")
print(f"(Set CITYPHI_MAZ_DATA and CITYPHI_TRIP_DATA environment variables to override)")

pandas.options.display.max_rows = 10

### Next Steps:

**Before running the cells below:**

**Data Setup Options:**
1. **Using relative paths (default):** Place your data in `cityphi_visualization/data/` folder:
   - `data/maz_shapes/` - MAZ shapefiles
   - `data/trips/` - trip CSV files (e.g., `indiv_trip.csv`)

2. **Using environment variables:** Set these before starting Jupyter:
   - `CITYPHI_MAZ_DATA` - path to MAZ shapefile directory
   - `CITYPHI_TRIP_DATA` - path to trip data directory

3. **Using absolute paths:** Edit the configuration cell above and uncomment Option 2

**After data is in place:**
1. Find the actual MAZ ID column name in your shapefile (run the first data cell to see available columns)
2. Update the `maz_id_column` variable in the centroid extraction cell

**Sections available:**
- ✅ MAZ Zones: Will visualize zone centroids
- ✅ Individual Trips: Will visualize trips at destination MAZs with time/purpose attributes
- ✅ Stacking, Coloring, Filtering: All work with trip data
- ⚠️ Polylines/Roads: SKIP - requires network export from EMME
- ⚠️ Trajectories: SKIP - requires GPS-like time-sequenced data
- ⚠️ Buildings/Polygons: SKIP - no building data available

You can run through the MAZ and Trips sections, then skip to the end.

In [ ]:
import cityphi.application
app = cityphi.application.Application()

There is some configuration that can be done on the app, such as changing the background color. This can be done in the CityPhi user interface, or in Python as shown below.

In [ ]:
app.background_color = (90, 90, 90)  # make the background grey
# turn on ambient occlusion
app.graphics_settings.ambient_occlusion = "SSAO_LOW"

## Web basemap

A web basemap layer is automatically added to the application when launched. This layer may be configured directly in the CityPhi user interface, by selecting the "Web Basemap" layer on the left.
Pick from one of the available basemaps.

The web basemap layer refreshes automatically based on the current view.
   
To simplify the view in the following examples you may wish to turn off the Web basemap or to lower the Opacity value.

In [ ]:
basemap_layer = app.layers[0]
basemap_layer.opacity = 20

## MAZ Zones Layer

A Layer is generated in two steps:
- Construct a Feature, which associates a set of features and their data with their geometries
- Construct and configure a layer, which displays the features

We'll start by loading the MAZ shapefile and extracting centroids to represent each zone as a point.

In [ ]:
# Load MAZ shapefile - find the .shp file
import glob
shp_files = glob.glob(os.path.join(maz_shapefile_path, "*.shp"))
print(f"Found shapefiles: {shp_files}")

# Load the shapefile (adjust filename as needed)
if shp_files:
    mazs_gdf = gpd.read_file(shp_files[0])
    print(f"Loaded {len(mazs_gdf)} MAZ zones")
    mazs_gdf.head()
else:
    print("No shapefile found! Please check the path.")

Now extract centroids from the MAZ polygons and prepare the coordinate data:

In [ ]:
# Extract centroids and prepare data for CityPhi
# Get MAZ IDs (adjust column name based on your shapefile)
# Common names: MAZ, MAZ_ID, ZONE_ID, etc.
print("Available columns:", mazs_gdf.columns.tolist())

# You'll need to update this with the actual MAZ ID column name
maz_id_column = mazs_gdf.columns[0]  # Temporary - replace with actual column
print(f"Using '{maz_id_column}' as MAZ ID")

# Calculate centroids in the original CRS
mazs_gdf['centroid'] = mazs_gdf.geometry.centroid

# Reproject to Web Mercator (EPSG:3857) for CityPhi
mazs_gdf_wm = mazs_gdf.to_crs(epsg=3857)

# Extract coordinates
maz_ids = mazs_gdf_wm[maz_id_column].values
centroids = mazs_gdf_wm['centroid'].to_crs(epsg=3857)
maz_points = numpy.array([[pt.x, pt.y, 0.0] for pt in centroids])

print(f"Prepared {len(maz_ids)} MAZ centroids")
print(f"Coordinate range: X({maz_points[:,0].min():.0f}, {maz_points[:,0].max():.0f}), "
      f"Y({maz_points[:,1].min():.0f}, {maz_points[:,1].max():.0f})")

#### PointFeature
We construct a layer of point data using a `PointFeature`. All feature objects are found in the `cityphi.feature` namespace. We will see other data types later.

In [ ]:
import cityphi.feature as _feat

In [ ]:
# Create arrays for CityPhi PointFeature
# maz_ids and maz_points are already prepared above
# Note: in pandas/numpy, .values returns corresponding Numpy representation

`PointFeature` needs:
- a numpy array of feature identifiers
- a numpy array of x,y,z coordinates.

Note: the numpy arrays must be of equal length - this is usually the case in CityPhi. It can be useful to check Numpy `shape` property to verify data layouts are correct

In [ ]:
parcel_ids.shape

In [ ]:
points.shape

In [ ]:
maz_pt_feat = _feat.PointFeature(maz_ids, maz_points)

#### Layer

In [ ]:
import cityphi.layer as _layer

A layer needs a `Feature` as input. Once the layer is created (and optionally configured, more about configuring layers soon), it can be added to the application using `add_layer()` method.

In [ ]:
maz_pt_layer = _layer.PointLayer(maz_pt_feat)
maz_pt_layer.name = "MAZ Centroids"
app.add_layer(maz_pt_layer)

To see the data, it is helpful to set the view to the dataset's full view. Execute the following code cell, or click the Full view button in CityPhi.

In [ ]:
view = app.full_view
app.set_view(view)

<img src="../images/simple_point.png" width="100%">

## Individual Trips Layer

We generate a layer of individual trips from the CTRAMP model output. Each trip represents a person traveling from one location to another. We will start by loading the trip data and exploring some basic layer configuration.
After we will see vertical stacks of this data under <a href="#Stacking-points">Stacking points</a>.

In [ ]:
# Load trip data
trips = pandas.read_csv(os.path.join(trip_data_path, "indiv_trip.csv"))
print(f"Loaded {len(trips)} trips")
trips.head()

Create a lookup dictionary to map destination MAZs to centroids:

In [ ]:
# Create MAZ to index lookup for linking trips to geometries
maz_to_idx = {maz_id: idx for idx, maz_id in enumerate(maz_ids)}

# Filter trips to only those with valid destination MAZs in our shapefile
trips_valid = trips[trips['dest_mgra'].isin(maz_to_idx.keys())].copy()
print(f"Trips with valid MAZ destinations: {len(trips_valid)} / {len(trips)}")

Convert time periods to minutes since midnight for CityPhi:

In [ ]:
# Convert stop_period (1-48) to minutes since midnight
# Period 1 = 3:00-3:29 AM = 180 minutes
# Each period = 30 minutes
trips_valid['start_time'] = (trips_valid['stop_period'] - 1) * 30 + 180
# Assume 30 minute duration for end_time (can be adjusted)
trips_valid['end_time'] = trips_valid['start_time'] + 30

# Handle wrap-around (periods after midnight)
trips_valid.loc[trips_valid['start_time'] >= 1440, 'start_time'] -= 1440
trips_valid.loc[trips_valid['end_time'] >= 1440, 'end_time'] -= 1440

print(f"Time range: {trips_valid['start_time'].min():.0f} to {trips_valid['start_time'].max():.0f} minutes")

We can reuse the maz_pt_feat `PointFeature` previously created in order to associate the trips which occur at each MAZ. This will enable the automatic stacking of trips later.
- Each trip is referenced to a particular MAZ by the *dest_mgra* column
 - There are many trips at each MAZ; a many-to-one relationship of data to physical geometry
 - `PointFeature.from_points` is used to construct a new point feature by referencing geometry defined in a base point feature.

`PointFeature.from_points` needs:  
- a numpy array of feature identifiers. This will be the trip ID (we'll create a unique ID)
- a numpy array of geometry identifiers. This will be the dest_mgra column, which indexes the MAZ feature IDs  (for the many-to-one correspondence)
- a `PointFeature` from which to look up geometry. In our case, this is the previously created maz_pt_feat

In relational database terms the *dest_mgra* column is used to join the trips DataFrame and maz_pt_feat Feature.

(We will use the *start_time*, *end_time*, and *dest_purpose* columns shortly.)

In [ ]:
# Create unique trip IDs and link to MAZ centroids
trips_valid = trips_valid.reset_index(drop=True)
trip_ids = trips_valid.index.values
dest_mazs = trips_valid.dest_mgra.values

trip_feat = _feat.PointFeature.from_points(
    trip_ids, dest_mazs, maz_pt_feat)

print(f"Created feature for {len(trip_ids)} trips")

In [ ]:
trip_layer = _layer.PointLayer(trip_feat)

In [ ]:
trip_layer.name = "Individual Trips"
app.add_layer(trip_layer)

### Layer ordering  
- There are now three layers, check with `app.layers`  

In [ ]:
app.layers

- Layer visibility can be toggled in the interface using the toggle switch located next to each layer name
- Alternatively, this can be done in code via `layer.visible = True|False`

In [ ]:
maz_pt_layer.visible = False
trip_layer.visible = True

Both layers are currently red. We will now change the color of the `trip_layer` to blue.

 - Note: colors may be specified as (red, green, blue) tuples, or as hex codes, e.g. `"#0000FF"`

In [ ]:
trip_layer.color = "#6f9af9"  # blue

In [ ]:
maz_pt_layer.visible = True

- Layer display order can be set by changing `layer.priority`
 - The priority is an index of the order in which the layers are displayed
 - Layers with a higher priority index are displayed on top of layers with a lower priority index
 - By default, priority is assigned in the order the layers are added; later layers appear on top of earlier layers
- Alternately, you can reorder layers in the interface by first enabling the "Reorder layers" checkbox, then dragging the layers up and down.

In [ ]:
# Trip layer under the MAZ point layer
trip_layer.priority = 1

In [ ]:
# Trip layer on top of the MAZ point layer
trip_layer.priority = 2

### Layer parameters and attributes
- CityPhi layers have a set of parameters that control display, e.g. color, height, width, min_pixel_size
- Some parameters, such as `min_pixel_size` apply globally to all features.
- Other parameters, such as `height` or `color`, can apply globally or can be different on a per-feature basis
 - A `FeatureAttribute` can be used for basic per-feature configuration - more on this under <a href="#Adding-feature-attributes">Adding feature attributes</a>
 - There are many predefined `Attribute` classes available in the `cityphi.attribute` namespace, which are well-suited for most use cases.
 - For more complex use cases, you can subclass `Attribute` and define your own `load` method.

In [ ]:
# Make trips a constant height 10m
trip_layer.height = 10

<img src="../images/extruded_point.png" width="100%">

### Stacking points

There are many trips at these MAZs, but we can't see them as they are drawn on top of each other at the same location.

To see how many, instead of drawing them over each other, we can stack them by setting the `stacked` layer parameter to `True`, with a 1m height for each trip.

In [ ]:
trip_layer.height = 1
trip_layer.stacked = True

<img src="../images/stacked_points.png" width="100%">

### Adding feature attributes
- The trip data has start and end times and trip purpose information.
- This data can be added as _feature_ attributes. They can then be used for colors, filters and queries, as we will see.  
- The trip purposes in the source data are strings. CityPhi does not yet have full support for string or categorical data, so for now we'll use integers to categorize the values using pandas.

In [ ]:
trips_valid.dest_purpose.unique()

In [ ]:
# Map destination purposes to integers
# Get unique purposes and create mapping
unique_purposes = trips_valid.dest_purpose.dropna().unique()
purposes = {x:i for i, x in enumerate(sorted(unique_purposes))}
print(f"Trip purposes: {list(purposes.keys())}")

trips_valid['p_type'] = trips_valid.dest_purpose.map(purposes).fillna(-1).astype(int)

The `feature.add_attribute` method needs:
- a name  
- a numpy data type  
- a numpy array of feature identifiers  
- a numpy array of attribute values

In [ ]:
trip_feat.add_attribute(
    "p_type", "int32", 
    trips_valid.index.values, 
    trips_valid.p_type.values)

### Coloring
- Layers have a _color_ parameter, which allows coloring individual features via a layer `Attribute`
- We can use the `DiscreteColorAttribute` to color by discrete classes of an `Attribute`
- We can use the `FeatureAttribute` to expose a feature attribute as an `Attribute`
- `DiscreteColorAttribute` needs:  
 - a list of (r,g,b) color tuples (or corresponding hex codes)
 - the layer attribute to color by,  
 - an optional list of labels for a color legend
 - an optional list of lists of values to associate to each color bin
- We will use the predefined colors available from the `colorbrewer` library

In [ ]:
# Prepare the colors and legends
import bokeh.palettes as _palettes

# Get colors based on number of purposes
n_purposes = len(purposes)
if n_purposes <= 12:
    purpose_colors = _palettes.Paired[max(3, n_purposes)][:n_purposes]
else:
    # Use a larger palette for more purposes
    purpose_colors = _palettes.Category20[min(20, n_purposes)][:n_purposes]

# Match the colors to the p_type in sort order
sorted_ps = sorted(purposes.items(), key=lambda x: x[1])
color_labels = [purpose[0] for purpose in sorted_ps]

list(zip(color_labels, purpose_colors))

In [ ]:
import cityphi.attribute as _att
p_type_attribute = _att.FeatureAttribute("p_type")
trip_layer.color = _att.DiscreteColorAttribute(
    purpose_colors, p_type_attribute, color_labels)

<img src="../images/coloring.png" width="100%">

- You can get a color legend by evaluating the `DiscreteColorAttribute`

In [ ]:
trip_layer.color

### Sorting

We can sort on any attribute, just as we can color by any attribute. We can also color and sort by the same attribute to see histogram-like stacks of purposes for each MAZ.

In [ ]:
trip_layer.sort_order = _att.FeatureAttribute("p_type")

If we wanted to sort in the reverse order (with the first purpose on top), we can create an attribute with negative p_type and sort by that attribute.

In [ ]:
trip_feat.add_attribute(
    "neg_p_type", "int32", 
    trips_valid.index.values, 
    (-1 * trips_valid.p_type.values))

trip_layer.sort_order = _att.FeatureAttribute("neg_p_type")

Or we can define a custom attribute to do this on-the-fly.

In [ ]:
class NegativePurposeAttribute(_att.Attribute):
    def load(self, feature_data):
        return -feature_data.p_type
trip_layer.sort_order = NegativePurposeAttribute()

To remove the sorting, set the `sort_order` to `None`. Or, set back to the original sort order.

In [ ]:
trip_layer.sort_order = None

In [ ]:
trip_layer.sort_order = _att.FeatureAttribute("p_type")

### Filtering
- What if instead of seeing all the trips, we are only interested in seeing specific trip purposes?
- We can do this by modifying the trip layer's `filter` parameter.
- This can be done interactively from the `Filter` button in the interface, or programmatically with a custom attribute as follows:

In [ ]:
# Show only first purpose type (adjust based on your data)
class TripFilter(_att.Attribute):
    def load(self, feature_data):
        return (feature_data.p_type == 0)
trip_layer.filter = TripFilter()

In [ ]:
# Show only second purpose type
class TripFilter(_att.Attribute):
    def load(self, feature_data):
        return (feature_data.p_type == 1)
trip_layer.filter = TripFilter()

In [ ]:
# Show only first or second purpose types
class TripFilter(_att.Attribute):
    def load(self, feature_data):
        return (feature_data.p_type == 0) | (feature_data.p_type == 1)
trip_layer.filter = TripFilter()

- We can go back to all trips by setting the `filter` parameter back to None

In [ ]:
trip_layer.filter = None

### Time Filtering

- All CityPhi layers come with native support for filtering by time.
- For trajectory data, this time filtering is implicit, but for other layers this can be enabled by setting the layer's `start_time` and/or `end_time` parameters, then playing with the layer's `time_window` parameter.
- We will start by adding the attributes for the start_time and end_time of the activities, and assign these to the respective parameters as feature attributes.

In [ ]:
trip_feat.add_attribute(
    "start_time", "int32", 
    trips_valid.index.values, 
    trips_valid.start_time.values)
trip_feat.add_attribute(
    "end_time", "int32", 
    trips_valid.index.values, 
    trips_valid.end_time.values)

trip_layer.start_time = _att.FeatureAttribute("start_time")
trip_layer.end_time = _att.FeatureAttribute("end_time")

- Then we simply assign a time window of interest to the layer's `time_window` parameter.

In [ ]:
# Trips occurring at 6:00 AM (period 7)
trip_layer.time_window = 360, 360  # 6:00 AM

In [ ]:
# Trips occurring before and up until 10:00AM
trip_layer.time_window = 180, 600  # 3:00 AM to 10:00 AM

In [ ]:
# Trips occurring at and after 10:00AM
trip_layer.time_window = 600, 180  # 10:00 AM onward

In [ ]:
# Trips occurring within 10:00AM and 12:00PM
trip_layer.time_window = 600, 720

Time filtering can be combined with a regular filter, in which case only those features which satisfy the filter, and which are occuring within the specified time window will be shown.

In [ ]:
# Show only specific trip purposes at specified time
class TripFilter(_att.Attribute):
    def load(self, feature_data):
        return (feature_data.p_type == 0) | (feature_data.p_type == 1)
trip_layer.filter = TripFilter()

In [ ]:
# All trip purposes
trip_layer.filter = None

<img src="../images/activities_time1300.png" width="100%">

## Interactivity
- Layer parameters (and other Python logic) can be connected to widgets in the CityPhi interface for interactivity
- Let's use a time slider to animate the time_window parameter

In [ ]:
import cityphi.widget as _widget

In [ ]:
# Set up time slider

# Current trips
def change_time(t):
    trip_layer.time_window = t, t

def format_time(t):
    return "%02d:%02d" % (t / 60, t % 60)
    
time_slider = _widget.TimeSlider(180, 1440, format_time, change_time)
app.add_widget(time_slider)

- Now when you drag or play the time slider, the trips will be time filtered to show only the current trips occurring at that time.
- We can change the time slider behavior by defining a new change_time function and assigning it to the time slider.

In [ ]:
# Cumulative trips
def change_time(t):
    trip_layer.time_window = 180, t
time_slider.time_callback = change_time

In [ ]:
# Trips within a 1 hour window of the current time
def change_time(t):
    trip_layer.time_window = t - 30, t + 30
time_slider.time_callback = change_time

- You can also define a new time slider and add it to the application, in which case the old time slider will be removed automatically.

In [ ]:
# Trips binned into 3 hour intervals
interval = 180

def change_time(t):
    t = interval * int(t / interval)
    trip_layer.time_window = t, t + interval
    
def format_time_binned(t):
    t = interval * int(t / interval)
    return format_time(t) + " - " + format_time(t + interval)

time_slider = _widget.TimeSlider(180, 1440, format_time_binned, change_time)
app.add_widget(time_slider)

In [ ]:
# Back to current trips

def change_time(t):
    trip_layer.time_window = t, t

def format_time(t):
    return "%02d:%02d" % (t / 60, t % 60)
    
time_slider = _widget.TimeSlider(180, 1440, format_time, change_time)
app.add_widget(time_slider)

## Queries
- CityPhi provides a way to query the current frame in order to quantify visualizations
- You can access feature attributes on the result with `.` notation

In [ ]:
# Set full view and time
view = app.full_view
app.set_view(view)
# Set the time slider's time programmatically
time_slider.time = 777  # 12:57

You can select features using the current camera view as a selection.

In [ ]:
# Query features visible within the current camera view
result = trip_layer.query_selection(app.camera)
print(f"Selected {len(result)} trips")

You can also select features using a box as a selection. Here we will define a box selection programmatically, and assign it to app.selection in order to visualize the selection area.

In [ ]:
import cityphi.query as _query
app.selection = _query.Box(
    (-13619268.621456677, 6042531.133449264, 0.0),
    (-13618623.231662149, 6042065.28818307, 0.0))

In [ ]:
# Query features within the box defined in app.selection
result = trip_layer.query_selection(app.selection)
print(f"Selected {len(result)} trips")

We can also select features using the box selection mode in the user interface, and query them interactively.

Activate the box selection mode in CityPhi (second button the toolbar), draw a box over some of the activities, and run the query below.

(Note: an error ``"AttributeError: 'NoneType' object has no attribute 'update'"``
will be shown if the selection was not yet set.)

In [ ]:
result = trip_layer.query_selection(app.selection)
print(f"Selected {len(result)} trips")

<img src="../images/selection.png" width="100%">

Now try re-evaluating the above cell after selecting a different group of features, or changing the current time by moving the time slider.

The current selection can be cleared with the fourth button on the toolbar.

It is also easy to plot distributions using the feature data
- Here we plot a histogram of trip purpose

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
result = trip_layer.query_selection(app.selection)
data = result.p_type
plt.hist(data, bins=range(0, len(purposes) + 1))

xlabels = sorted(purposes, key=purposes.get)
locs = [x + 0.5 for x in range(0, len(purposes))]
locs, labels = plt.xticks(locs, xlabels)
plt.setp(labels, rotation=45)
plt.show()

It is useful to be able to access data that is not available as a CityPhi feature attribute, but which exists in the original data, for instance in a pandas DataFrame.
- CityPhi features include the `feature_id` attribute, which corresponds to the identifiers that were used to construct the feature.

In [ ]:
result = trip_layer.query_selection(app.selection)
result.feature_id

- `feature_id` can be used to select pandas rows using the column used as the feature identifiers:
 - this can be accessed using `.loc` on the pandas dataframe

In [ ]:
result = trip_layer.query_selection(app.selection)
matching_result = trips_valid.loc[result.feature_id]
matching_result

Note that queries will also respond to changing filters, as below: 

In [ ]:
# Filter by trip purpose
class TripPurposeFilter(_att.Attribute):
    def load(self, feature_data):
        return (feature_data.p_type == 3) | (feature_data.p_type == 4)

trip_layer.filter = TripPurposeFilter()

In [ ]:
result = trip_layer.query_selection(app.selection)
data = result.p_type
plt.hist(data, bins=range(0, len(purposes) + 1))

xlabels = sorted(purposes, key=purposes.get)
locs = [x + 0.5 for x in range(0, len(purposes))]
locs, labels = plt.xticks(locs, xlabels)
plt.setp(labels, rotation=45)
plt.show()

In [ ]:
# Go back to all trips
trip_layer.filter = None

## Polylines

Polyline data can be brought into CityPhi to (for example) show roads.
- First, read in data for the OSM roads (extracted from the 'ways')

In [ ]:
road_data_df = pandas.read_csv(os.path.join(data_path, "road_data.csv"))

In [ ]:
roads_df = pandas.read_csv(os.path.join(data_path, "road_shapes.csv"))
roads_df

The basic `PolylineFeature` constructor needs:  
- a numpy array of feature identifiers  
- a numpy array of x,y,z coordinates  
- a numpy array of sequence numbers

The polylines will be built by grouping the records with the same feature identifier and sequencing the coordinates using the sequence numbers.

In [ ]:
point_road_ids = roads_df.road_id.values
points = roads_df[['x', 'y', 'z']].values
point_seq = roads_df.shape_pt_seq.values

road_feature = _feat.PolylineFeature(point_road_ids, points, point_seq)

Create the layer, customize its look, and add it to CityPhi.

In [ ]:
road_layer = _layer.PolylineLayer(road_feature)

In [ ]:
road_layer.name = "Roads"
road_layer.width = 5.0
road_layer.min_pixel_size = 1.0
road_layer.color = (150, 150, 150)

In [ ]:
app.add_layer(road_layer)
basemap_layer.visible = False

<img src="../images/polylines.png" width="100%">

### Animated flow on polylines

One way of showing speed and flow in CityPhi is with an animated flow layer. We have the road type saved with the road data:

In [ ]:
road_data_df

The `type` corresponds to a Highway category of the scheme shown in the `HIGHWAY_TYPES` dictionary below. We will bring this data over to CityPhi.

In [ ]:
HIGHWAY_TYPES = {
    "motorway" : 1, 
    "trunk" : 2, 
    "primary" : 3, 
    "secondary" : 4,
    "tertiary" : 5,
    "unclassified" : 6, 
    "residential" : 7, 
    "service" : 8
}

road_feature.add_attribute("type", "int32", road_data_df.road_id.values,
                           road_data_df.type.values)

Here we create a `FlowLayer`, which takes as input a polyline feature, very similar to a `PolylineLayer`. A few cells below we configure the animation parameters - which may be different per polyline.

In [ ]:
flow_layer = _layer.FlowLayer(road_feature)
flow_layer.name = "Flow"

... and filter out the 'unclassified' (type 6) and 'service' roads (type 8) from the `flow_layer`.

In [ ]:
class WaysFilter(_att.Attribute):
    def load(self, feature_data):
        return (feature_data.type != 6) & (feature_data.type != 8)

flow_layer.filter = WaysFilter()

Set the flow and duration for the flow_layer.
- flow: the number of vehicles (or equivalent items) per unit time
- duration: the time to traverse the polyline (or link)

The speed is determined from the duration and the length of the polyline.
In our case we will set the same value on all polylines to show a simple animation.

Try tweaking these values.

In [ ]:
flow_layer.flow = 0.2
flow_layer.duration = 10

Configure the layer display

In [ ]:
flow_layer.radius = 10
flow_layer.color = "#4bb4c9"
flow_layer.min_pixel_size = 3
flow_layer.glowing = True
flow_layer.min_pixel_size = 20
flow_layer.alignment = "RIGHT"

In [ ]:
app.add_layer(flow_layer)

Connect the timeslider to the flow_layer

In [ ]:
def change_time(t):
    act_layer.time_window = t, t
    flow_layer.time = t

time_slider.time_callback = change_time

Now run the animation. We just may want to turn off the parcel points, tweak web basemap settings and change the view by running the following cell:

In [ ]:
parcel_pt_layer.visible = False
basemap_layer.visible = True
basemap_layer.max_magnification = 2.0
basemap_layer.opacity = 21
basemap_layer.basemap = cityphi.basemap.OPEN_STREET_MAP
app.background_color = "#000000"
app.camera.set_current_view = {'distance': 2753.16506915602,
                               'rotation': (358.0, 31.0),
                               'translation': (-13618996.211108863, 6041932.698972598)}

<img src="../images/animated_flow_new.png" width="100%">

### Extruded polylines
We can add heights to our polylines. This can be used to simply modify the visual, showing 3-dimensional items, or for representing data such as flow. Any polyline layer can be extruded - to demonstrate we will create a new layer with the road data.

In [ ]:
highways_layer = _layer.PolylineLayer(road_feature)
highways_layer.name = "Highways"

Add a filter to show only the highway polylines (type value of 1)

In [ ]:
class HighwaysFilter(_att.Attribute):
    def load(self, feature_data):
        return (feature_data.type == 1)

highways_layer.filter = HighwaysFilter()

Set the alignment of the extrusion, `"LEFT"`, `"RIGHT"` or `"CENTER"`, and a width in meters.

In [ ]:
highways_layer.alignment = "RIGHT"   # or "LEFT" or "CENTER"
highways_layer.width = 5

Configure the layer display

In [ ]:
highways_layer.height = 20
highways_layer.color = (136, 163, 204)

In [ ]:
app.add_layer(highways_layer)

## Trajectories
CityPhi can also show animated trajectory data. A trajectory is defined as a sequence of sampled locations and times (x, y, z, t) 

The sample data shows several buses which converge downtown around 7:30 am.

In [ ]:
trajectories_df = pandas.read_csv(os.path.join(data_path, "trajectories.csv"))
trajectories_df

The basic `TrajectoryFeature` constructor needs:
 - a numpy array of feature identifiers
 - a numpy array of point x, y, z coordinates
 - a numpy array of times

The trajectories will be built by grouping the records with the same feature identifier and sequencing the coordinates by increasing times.

In [ ]:
vehicle_trip_ids = trajectories_df.trip_id.values
vehicle_points = trajectories_df[["x", "y", "z"]].values
vehicle_times = trajectories_df.time.values

vehicle_feature = _feat.TrajectoryFeature(
    vehicle_trip_ids, vehicle_points, vehicle_times)

### Animated points
Here, we'll create an `AnimatedPointLayer` to visualize the trajectories

In [ ]:
vehicle_layer = _layer.AnimatedPointLayer(vehicle_feature)
vehicle_layer.name = "Vehicles"
app.add_layer(vehicle_layer)

- Some layer customization

In [ ]:
vehicle_layer.glowing = True
vehicle_layer.min_pixel_size = 40.0
vehicle_layer.color = (255, 240, 86) # A relatively bright yellow
app.background_color = (25, 25, 25)  # make the background black / dark gray

Finally let's add some animation so we can see the trajectories moving.
- Update the `change_time` function to also change the time parameter on the `vehicle_layer`
- The activities layer from above is also being animated

Remember, trajectories can be fleeting! They pass through the visible network between 7:15am and 7:50am

In [ ]:
def change_time(t):
    act_layer.time_window = t, t
    flow_layer.time = t
    vehicle_layer.time = t

time_slider.time_callback = change_time
time_slider.pause()

In [ ]:
# Jump to 7:15 am to watch the vehicle animation
# Zoom out to see the full trajectories
time_slider.time = 435
time_slider.play()
app.camera.set_view({'distance': 3398.9692211802712,
                     'rotation': (359.0, 30.0),
                     'translation': (-13618189.092867726, 6042003.390481629)})


<img src="../images/trajectories_new.png" width="100%">

### Query by screenline

CityPhi provides a way to query features crossing screenlines in the `AnimatedPointLayer`, `TrajectoryLayer` and `PolylineLayer`.

Here we'll define and use a screenline programmatically, and assign it to app.selection in order to visualize the screenline.

In [ ]:
app.selection = _query.Line(
    (-13618458.244797919, 6042264.242932153, 0.0),
    (-13618725.135315863, 6042104.108622895, 0.0))

We issue a screenline query by calling ``layer.query_screenline``, providing the selection, a time window, and whether we want one total result (``directional=False``) or one for each crossing direction (``directional=True``).

The time window can be set to ``(0, 1440)`` to catch anyone crossing at any time, or we can specify 0 to current time to capture only those that have already crossed, as in the example below.

In [ ]:
result = vehicle_layer.query_screenline(
    app.selection, (0, time_slider.time), directional=False)
result.feature_id

You can also use the *Screenline* tool to draw a line. Activate the screenline selection mode in CityPhi (third button the toolbar), and draw a line crossing an area of interest, and run the query below.

In [ ]:
result = vehicle_layer.query_screenline(
    app.selection, (0, time_slider.time), directional=False)
result.feature_id

Below we will use a screenline select vehicles, and color the vehicles either white (not selected) or green (selected).

In [ ]:
import cityphi.parameter as _param

class SelectedVehiclesAttribute(_att.Attribute):

    @_param.ListParameter(int)
    def vehicles(self, value):
        self._vehicles = numpy.array(value)

    def load(self, feature_data):
        return numpy.in1d(feature_data.feature_id, self._vehicles)

selected_vehicles = SelectedVehiclesAttribute()
selected_vehicles.vehicles = []
vehicle_layer.color = _att.DiscreteColorAttribute(
    [(255, 255, 255), (0, 255, 0)], selected_vehicles)

Here, we have defined our own _vehicles_ parameter on the custom attribute, which stores the selected vehicles. By defining it as a parameter, we inform CityPhi that a change to _vehicles_ implies the `SelectedVehiclesAttribute` has changed, and hence the color has changed.

In [ ]:
result = vehicle_layer.query_screenline(
    app.selection, (0, time_slider.time), directional=False)
selected_vehicles.vehicles = result.feature_id

In [ ]:
result = vehicle_layer.query_screenline(
    app.selection, vehicle_feature.time_range, directional=True)
selected_vehicles.vehicles = result[0].feature_id   # or result[1].feature_id

### Parabolic trajectories
Trajectories can also be visualized as parabolic arcs using the `ParabolicTrajectoryLayer`. In this case, a parabolic arc is used to represent each trajectory, and is animated from the first point/time to the last point/time, ignoring any intermediates.

In [ ]:
parabolic_layer = _layer.ParabolicTrajectoryLayer(vehicle_feature)

In [ ]:
parabolic_layer.name = "Parabolas"
parabolic_layer.color = (157, 253, 146)
parabolic_layer.section_width = 10
parabolic_layer.section_height = 5
parabolic_layer.min_pixel_size = 3
parabolic_layer.height = 1500

In [ ]:
app.add_layer(parabolic_layer)

The parabolic trajectories are animated with a `time_window`. We can set the time window to include the entire day to see all the trajectories at once.

In [ ]:
parabolic_layer.time_window = (0, 1400)

... or to a 20 minute window to see the portions of the trajectory for just that time.

In [ ]:
parabolic_layer.time_window = (430, 450) # 7:10 to 7:30

We can animate the parabolic trajectories by setting the `time_window` to a moving interval. 
Try changing the time window below, `parabolic_layer.time_window = (t - 5, t)` to a different range. (The form `(t - x, t)` aligns the head of the moving parabola with the current time, and the tail with 5 minutes before.)

In [ ]:
def change_time(t):
    act_layer.time_window = t, t
    flow_layer.time = t
    vehicle_layer.time = t
    parabolic_layer.time_window = (t - 10, t)

time_slider.time_callback = change_time

<img src="../images/parabolic_trajectories_new.png" width="100%">

### Motion trails

Yet another way to visualize trajectories is as animated polylines, using the `TrajectoryLayer`.

In [ ]:
motion_trail_layer = _layer.TrajectoryLayer(vehicle_feature)
motion_trail_layer.name = "Motion trails"
app.add_layer(motion_trail_layer)

In [ ]:
motion_trail_layer.color = vehicle_layer.color
motion_trail_layer.width = vehicle_layer.radius
motion_trail_layer.priority = 6

We can animate the trajectories as motion trails by setting the layer's `time_window` to a moving interval, as with the parabolic trajectories.

In [ ]:
def change_time(t):
    act_layer.time_window = t, t
    flow_layer.time = t
    vehicle_layer.time = t
    parabolic_layer.time_window = (t - 5, t)
    motion_trail_layer.time_window = (t - 1, t)

time_slider.time_callback = change_time

- The trajectory layer also offers a _glowing_ rendering style.
- The _glowing_ style renders the trajectories like light sources, which go from fully bright to fully dark between the end and start of the `time_window`

In [ ]:
motion_trail_layer.glowing = True

The color of overlapping glowing trajectories will blend together the way that light does. Here we show this effect by setting a longer time window and tweaking the selection coloring.

In [ ]:
def change_time(t):
    act_layer.time_window = t, t
    flow_layer.time = t
    vehicle_layer.time = t
    parabolic_layer.time_window = (t - 5, t)
    motion_trail_layer.time_window = (t - 10, t)

time_slider.time_callback = change_time
motion_trail_layer.color.colors = [(216, 19, 19), (0, 78, 255)]

## Polygons

Polygon data can be used in CityPhi to show zone areas, or in the example below, building footprints. Polygons are constructed from the same basic geometry information as polylines.

The basic PolygonFeature constructor needs:
 - a numpy array of feature identifiers
 - a numpy array of x,y,z coordinates
 - a numpy array of sequence numbers

Only simple polygons - polygons without holes - are supported at the current version.  
To draw a polygon with an outline and fill, create separate polygon and polyline layers from the same polygon feature, and configure them accordingly.

In [ ]:
buildings_df = pandas.read_csv(os.path.join(data_path, "building_shapes.csv"))
building_data_df = pandas.read_csv(os.path.join(data_path, "building_data.csv"))

In [ ]:
building_pt_ids = buildings_df.building_id.values
building_pts = buildings_df[["x", "y", "z"]].values
building_pt_seq = buildings_df.shape_pt_seq.values

building_feature = _feat.PolygonFeature(
    building_pt_ids, building_pts, building_pt_seq)

In [ ]:
building_layer = _layer.PolygonLayer(building_feature)
building_layer.name = "Buildings"
building_layer.color = (131, 137, 155)

In [ ]:
app.add_layer(building_layer)

The polygons can be extruded by setting the `height` parameter. In this example, we will add building height as a feature attribute and use this to represent the shape of the city, providing context for the rest of the animation.

We will also turn off the activities layer so that the view is less cluttered.

Note: in the original source data, not all building footprints included height, so a default height of 15 meters was used for those.

In [ ]:
act_layer.visible = False

building_feature.add_attribute(
    "height", "float64", 
    building_data_df.building_id.values, 
    building_data_df.height.values)
building_layer.height = _att.FeatureAttribute("height")

The light direction can also be changed to have the sun (light-source) coming from the west.

In [ ]:
default_direction = app.light.direction

In [ ]:
app.light.direction = (-1, -0.8, 0.7)

In [ ]:
# Restore the default light direction
app.light.direction = default_direction

For fun, you can turn on a _tilt shift_ effect to give the illusion that this is a miniature city.

In [ ]:
app.graphics_settings.tilt_shift = True

<img src="../images/buildings_and_all.png" width="100%">

## Video Recording

You can use the video recording tool on the application top bar to try out the two recording modes. 

__Interactive__ mode will let you record interactive, hands-on use of the software including user interface details. 

__Presentation__ mode records a video where time and the camera are automated hands-free, and guarantees smooth production quality playback. You can set up your own keyframes for camera automation using the _Editor mode_ button on the right-side of the time slider, or you can run the following cell to load up keyframes for playback. When you record in Presentation mode, the camera will be automatically synchronized with the keyframes, or else will remain stationary. If you want to preview the keyframes without recording video, make sure the _Sync with keyframes_ time slider button is depressed. You can use the time slider _speed_ to speed up or slow down the video.

If you'd like to load up keyframes that we've already prepared, and set a nice recording speed, run the cell below. 

In [ ]:
time_slider.tracks[0].keyframes = \
[{u'data': {u'distance': 4196.258297753424,
   u'rotation': [359, 38],
   u'translation': [-13618228.087056464, 6041300.633806811]},
  u'end_time': 439.75130890052355,
  u'start_time': 433.9714923867621},
 {u'data': {u'distance': 1806.351601873266,
   u'rotation': [305, 58],
   u'translation': [-13619309.32835269, 6042668.204551189]},
  u'end_time': 459.4602869013877,
  u'start_time': 454.50615846102073},
 {u'data': {u'distance': 1806.351601873266,
   u'rotation': [35, 49],
   u'translation': [-13618245.912473815, 6042001.280695734]},
  u'end_time': 486.0518465344138,
  u'start_time': 482.7490942408358}]

time_slider.speed = 4

Now go ahead and use CityPhi Studio to record a Presentation Mode video, by pressing the record button on the video recording window, like this: <img src="../images/recording.png" width="100%">. <br> When the recording is done, you can find the video from the pop-up notification at the bottom-right of the screen: <img align="right" src="../images/recorded.png">